In [85]:
workspace()

In [86]:
type Transition
    check::Function
    destination
end

type Behavior
    args
    action::Function
    transitions::Vector{Transition}
end

In [106]:
macro Behavior(args, action)
    return quote
        Behavior(args, $(args) -> $(action), [])
    end
end

macro add_transition!(behavior, dest, args, body)
    return quote
        t = Transition(esc($(plus.args).args -> $(body), $(dest)))
        push!($(behavior).transitions, t)
    end
end

function Behavior(args, body)
    @eval Behavior(esc(args), $(args) -> $(body), [])
end

Behavior

In [107]:
plus = Behavior(:(x::Float64, y), :(x + y))

LoadError: LoadError: UndefVarError: args not defined
while loading In[107], in expression starting on line 1

In [90]:
plus = @Behavior (x::Float64, y) x + y
minus = @Behavior (x::Float64, y) x - y

LoadError: LoadError: UndefVarError: args not defined
while loading In[90], in expression starting on line 3

In [101]:
plus.action

(anonymous function)

In [80]:
macroexpand(:(@add_transition! plus minus plus.args x > 3))

:($(Expr(:error, "\"Array{Any, 1}[Expr(:::, :x, :Float64)::Any, :y].args\" is not a valid function argument name")))

In [84]:
@eval $(plus.args) -> x

LoadError: LoadError: syntax: "Array{Any, 1}[Expr(:::, :x, :Float64)::Any, :y]" is not a valid function argument name
while loading In[84], in expression starting on line 1

In [12]:
dump(expand(:((x::Float64, y) -> x + y)).ast.args[2])

Array(Any,(4,))
  1: Array(Any,(2,))
    1: Array(Any,(3,))
      1: Symbol x
      2: Symbol Any
      3: Int64 18
    2: Array(Any,(3,))
      1: Symbol y
      2: Symbol Any
      3: Int64 0
  2: Array(Any,(0,))
  3: Int64 0
  4: Array(Any,(0,))


In [6]:
expand(:(f))

:f